In [ ]:
# Uncomment to install autocoder if not installed
# !pip install git@github.com:ttumiel/autocoder.git

# Autocoder

What does it do?
- Automatically generate JSON schemas of python functions
- Integrate into ChatGPT plugins
- Automate function calling in python
- Convert unstructured data to structured data

In [ ]:
from pprint import pprint as print
from autocoder import json_schema


@json_schema
def addition(x: float, y: float) -> float:
    """adds two numbers

    Args:
        x (float): first number
        y (float): second number
    """
    return x + y

print(addition.json)

# Make it a Plugin

In [ ]:
from autocoder.server import FunctionServer

server = FunctionServer({"addition": addition})
server.run(debug=False, use_reloader=False) # use_reloader=False for Jupyter

# Using the Function Calling API

**Converting Unstructured Data to Structured Data**

_(I was writing this code just last night so it's kinda janky :))_

In [ ]:
# Get the imports!
from dataclasses import dataclass
from typing import Optional
from autocoder import function_call
from autocoder.api import FunctionCallingAPI, Role


@json_schema
@dataclass
class Character:
    name: str
    age: Optional[int] = None
    house: Optional[str] = None

In [ ]:
# Setup our function calling API
system_prompt = "You are a literary expert on the Dune book series."
functions = {"Character": Character}
api = FunctionCallingAPI(functions, system_prompt)
api.chat.add_message(Role.USER.value, "List the characters in the Dune series. Give best estimates of age's where appropriate.")

In [ ]:
for _ in range(5):
    # Force the API to call a function
    message = api.reply(force_function="Character")
    api.chat.messages.append(message)

    # Call the function (and validate the inputs!)
    fn_name = message.function_call["name"]
    args = message.function_call["arguments"]
    result = function_call(fn_name, args, functions, return_json=False)

    # Add the result to the chat
    api.chat.add_message(Role.FUNCTION.value, str(result), name="character")
    print(result)

# Deploy!

It's just a flask app. You could use a simple google cloud function to deploy it.








.   